In [8]:
import geopandas as gpd
import rasterio
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
import os
import pandas as pd

# Paths
shp_path = "./area/jade.shp"
raster_dir = "./tiffs_reprojected"
output_path = "./jade_profile.csv"

# Step 1: Read the shapefile and inspect its contents
gdf = gpd.read_file(shp_path)

# Check the number of features and geometry types
print(f"Number of features in shapefile: {len(gdf)}")
print(f"Geometry types: {gdf.geom_type.unique()}")

# Step 2: Handle single or multiple lines
lines = []
if len(gdf) == 1:
    # Single feature
    geom = gdf.geometry.iloc[0]
    if isinstance(geom, LineString):
        lines.append(geom)
    elif isinstance(geom, MultiLineString):
        lines.extend(geom.geoms)  # Extract individual LineStrings from MultiLineString
    else:
        raise ValueError("Unexpected geometry type in shapefile")
else:
    # Multiple features
    for geom in gdf.geometry:
        if isinstance(geom, LineString):
            lines.append(geom)
        elif isinstance(geom, MultiLineString):
            lines.extend(geom.geoms)
        else:
            raise ValueError("Unexpected geometry type in shapefile")

print(f"Number of LineString geometries to process: {len(lines)}")

# Step 3: Generate points along each line at ~101m intervals
all_points = []
all_distances = []
line_id = []

for idx, line in enumerate(lines):
    line_length = line.length
    distances = np.arange(0, line_length, 101)  # 101m intervals
    points = [line.interpolate(distance) for distance in distances]
    all_points.extend(points)
    all_distances.extend(distances)
    line_id.extend([idx] * len(distances))  # Track which line the points belong to

# Step 4: List all raster files
raster_files = [f for f in os.listdir(raster_dir) if f.endswith(".ti.geo.tif")]
raster_files.sort()  # Sort by date

# Step 5: Extract raster values for each point
results = {
    "line_id": line_id,
    "distance_from_start": all_distances
}
for raster_file in raster_files:
    date = raster_file.split(".")[0]  # Extract date (e.g., 20150220)
    with rasterio.open(os.path.join(raster_dir, raster_file)) as src:
        values = []
        for point in all_points:
            # Get raster value at point
            x, y = point.x, point.y
            try:
                value = next(src.sample([(x, y)]))[0]
                values.append(value if not np.isnan(value) else np.nan)
            except:
                values.append(np.nan)  # Handle cases where point is outside raster
        results[f"val_{date}"] = values

# Step 6: Create a DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv(output_path, index=False)
print(f"Output saved to {output_path}")

Number of features in shapefile: 1
Geometry types: ['LineString']
Number of LineString geometries to process: 1
Output saved to ./jade_profile.csv


In [3]:
import geopandas as gpd
import rasterio
import numpy as np
from shapely.geometry import Point
import os
import pandas as pd

# Paths
shp_path = "./highway/بزرگراه قوچان مشهد.shp"
raster_dir = "./tiffs_reprojected"
output_path = "./highway_profile_with_xy.csv"

# Step 1: Read the shapefile
gdf = gpd.read_file(shp_path)

# Verify and reproject CRS to UTM
print(f"Original Shapefile CRS: {gdf.crs}")
if gdf.crs != "EPSG:32640":  # Assuming UTM Zone 40N for Mashhad, Iran; adjust if needed
    gdf = gdf.to_crs(epsg=32640)  # Reproject to UTM Zone 40N
    print(f"Reprojected Shapefile CRS: {gdf.crs}")

# Ensure the CRS is projected
if not gdf.crs.is_projected:
    raise ValueError("Shapefile CRS is not projected (UTM) after reprojection.")

# Step 2: Get the line geometry
line = gdf.geometry.iloc[0]  # Assuming the shapefile contains a single line

# Step 3: Get start and end points
start_point = line.coords[0]  # First vertex
end_point = line.coords[-1]   # Last vertex
print(f"Start Point (X, Y in UTM): {start_point}")
print(f"End Point (X, Y in UTM): {end_point}")

# Step 4: Generate points along the line at ~101m intervals
line_length = line.length
distances = np.arange(0, line_length, 101)  # 101m intervals
points = [line.interpolate(distance) for distance in distances]

# Step 5: Extract X, Y coordinates for each point
x_coords = [point.x for point in points]
y_coords = [point.y for point in points]

# Step 6: List all raster files
raster_files = [f for f in os.listdir(raster_dir) if f.endswith(".ti.geo.tif")]
raster_files.sort()  # Sort by date

# Step 7: Extract raster values for each point
results = {
    "distance_from_start": distances,
    "X_UTM": x_coords,
    "Y_UTM": y_coords
}
for raster_file in raster_files:
    date = raster_file.split(".")[0]  # Extract date (e.g., 20150220)
    with rasterio.open(os.path.join(raster_dir, raster_file)) as src:
        # Ensure raster CRS matches shapefile CRS
        if src.crs != gdf.crs:
            print(f"Warning: Raster {raster_file} CRS ({src.crs}) does not match shapefile CRS ({gdf.crs}).")
        values = []
        for point in points:
            x, y = point.x, point.y
            try:
                value = next(src.sample([(x, y)]))[0]
                values.append(value if not np.isnan(value) else np.nan)
            except:
                values.append(np.nan)  # Handle cases where point is outside raster
        results[f"val_{date}"] = values

# Step 8: Create a DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv(output_path, index=False)
print(f"Output saved to {output_path}")

# Optional: Print first few rows of the DataFrame
print("\nFirst few rows of the output:")
print(df.head())

Original Shapefile CRS: EPSG:4326
Reprojected Shapefile CRS: EPSG:32640
Start Point (X, Y in UTM): (691685.8771544565, 4055596.810697063)
End Point (X, Y in UTM): (704177.0013407576, 4046116.562995446)
Output saved to ./highway_profile_with_xy.csv

First few rows of the output:
   distance_from_start          X_UTM         Y_UTM  val_20150220  \
0                  0.0  691685.877154  4.055597e+06           0.0   
1                101.0  691768.965296  4.055539e+06           0.0   
2                202.0  691848.206590  4.055477e+06           0.0   
3                303.0  691930.328072  4.055418e+06           0.0   
4                404.0  692015.131554  4.055363e+06           0.0   

   val_20150304  val_20150328  val_20150421  val_20150515  val_20150702  \
0     -1.764547     -2.822982     -4.353869     -4.746088      1.194003   
1     -1.720154     -2.967121     -4.773423     -5.346199      0.338164   
2     -1.580621     -2.927372     -4.618943     -4.576911      1.048642   
3     

In [ ]:
import geopandas as gpd
import requests
import time
from datetime import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Tuple, List
import logging
from pathlib import Path

# --- Configure Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# --- Constants ---
UTM_EPSG = 32643
WGS84_EPSG = 4326
BUFFER_WIDTH = 1000
INTERVAL_M = 100
OSM_RADIUS = 1000
OVERPASS_URLS = [
    "http://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter"
]
RETRY_ATTEMPTS = 3
RETRY_DELAY = 5  # Increased delay for retries (seconds)
REQUEST_TIMEOUT = 30
OUTPUT_DIR = Path("output")
MAX_WORKERS = 3  # Reduced for stability

# --- Setup Output Directory ---
OUTPUT_DIR.mkdir(exist_ok=True)

def setup_timer() -> datetime:
    return datetime.now()

def load_shapefile(filepath: str) -> gpd.GeoDataFrame:
    try:
        gdf = gpd.read_file(filepath)
        if gdf.empty:
            raise ValueError("Shapefile is empty.")
        if gdf.geometry.iloc[0].geom_type != 'LineString':
            raise ValueError("Shapefile must contain a LineString geometry.")
        return gdf
    except Exception as e:
        logger.error(f"Error loading shapefile: {e}")
        raise

def reproject_to_utm(gdf: gpd.GeoDataFrame, epsg: int) -> gpd.GeoDataFrame:
    if gdf.crs.to_epsg() != epsg:
        logger.info(f"Reprojecting to EPSG:{epsg}")
        return gdf.to_crs(epsg=epsg)
    return gdf

def generate_points_along_line(line, interval_m: float = INTERVAL_M) -> Tuple[List, List]:
    points, distances = [], []
    length = line.length
    steps = int(length // interval_m) + 1
    for i in range(steps):
        dist = i * interval_m
        points.append(line.interpolate(dist))
        distances.append(dist)
    return points, distances

def create_buffer_gdf(points: List, distances: List, gdf_crs, buffer_width: float = BUFFER_WIDTH) -> gpd.GeoDataFrame:
    points_gdf = gpd.GeoDataFrame({'distance_m': distances}, geometry=points, crs=gdf_crs)
    buffers = points_gdf.buffer(buffer_width / 2)
    buffer_centroids = buffers.centroid
    return gpd.GeoDataFrame({'distance_m': distances}, geometry=buffer_centroids, crs=gdf_crs)

def build_overpass_query(lat: float, lon: float, radius: float = OSM_RADIUS) -> str:
    return f"""
    [out:json];
    (
      node["place"~"village|hamlet|town"](around:{radius},{lat},{lon});
      way["natural"="water"](around:{radius},{lat},{lon});
      way["waterway"="river"](around:{radius},{lat},{lon});
      relation["natural"="water"](around:{radius},{lat},{lon});
      way["highway"](around:{radius},{lat},{lon});
      way["leisure"~"park|garden"](around:{radius},{lat},{lon});
      way["landuse"="grass"](around:{radius},{lat},{lon});
    );
    out center;
    """

def query_osm_features(point, radius: float = OSM_RADIUS) -> List[str]:
    lat, lon = point.y, point.x
    query = build_overpass_query(lat, lon, radius)
    
    for url_idx, overpass_url in enumerate(OVERPASS_URLS):
        for attempt in range(RETRY_ATTEMPTS):
            try:
                response = requests.get(overpass_url, params={'data': query}, timeout=REQUEST_TIMEOUT)
                time.sleep(RETRY_DELAY)  # Increased delay
                if response.status_code == 502:
                    logger.warning(f"Attempt {attempt + 1} failed with status 502 at ({lat}, {lon}) using {overpass_url}")
                    continue
                if response.status_code != 200:
                    logger.warning(f"Attempt {attempt + 1} failed with status {response.status_code} at ({lat}, {lon})")
                    continue
                data = response.json()
                features = []
                for el in data.get('elements', []):
                    tags = el.get('tags', {})
                    name = tags.get('name', 'Unnamed')
                    if 'place' in tags:
                        features.append(f"Place: {name}")
                    elif tags.get('waterway') == 'river':
                        features.append(f"River: {name}")
                    elif tags.get('natural') == 'water':
                        features.append(f"Water body: {name}")
                    elif tags.get('highway'):
                        features.append(f"Road: {name}")
                    elif tags.get('leisure') in ['park', 'garden']:
                        features.append(f"{tags['leisure'].capitalize()}: {name}")
                    elif tags.get('landuse') == 'grass':
                        features.append(f"Green area: {name}")
                return features if features else ["None"]
            except Exception as e:
                logger.warning(f"Attempt {attempt + 1} failed at ({lat}, {lon}) using {overpass_url}: {e}")
                if attempt == RETRY_ATTEMPTS - 1 and url_idx == len(OVERPASS_URLS) - 1:
                    logger.error(f"Exhausted retries for point ({lat}, {lon})")
                    return [f"[Error] Failed after {RETRY_ATTEMPTS} attempts: {e}"]
    return ["[Error] All servers failed"]

def parallel_osm_queries(points_gdf: gpd.GeoDataFrame, max_workers: int = MAX_WORKERS) -> List:
    features_list = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {executor.submit(query_osm_features, row.geometry): idx 
                        for idx, row in points_gdf.iterrows()}
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                features = future.result()
                features_list.append((idx, "; ".join(features)))
            except Exception as e:
                logger.error(f"Error processing point {idx}: {e}")
                features_list.append((idx, f"[Error] {e}"))
    features_list.sort(key=lambda x: x[0])
    return [f for _, f in features_list]

def save_outputs(gdf: gpd.GeoDataFrame, geojson_file: str, csv_file: str) -> None:
    try:
        gdf.to_file(geojson_file, driver='GeoJSON')
        gdf[['distance_m', 'features_nearby']].to_csv(csv_file, index=False, encoding='utf-8-sig')
        logger.info(f"Saved outputs to {geojson_file} and {csv_file}")
    except Exception as e:
        logger.error(f"Error saving outputs: {e}")
        raise

def main():
    start_time = setup_timer()
    route_fp = 'خط مترو.shp'
    route_gdf = load_shapefile(route_fp)
    route_gdf = reproject_to_utm(route_gdf, UTM_EPSG)
    route_line = route_gdf.geometry.iloc[0]
    points, distances = generate_points_along_line(route_line)
    buffers_gdf = create_buffer_gdf(points, distances, route_gdf.crs)
    buffers_wgs = buffers_gdf.to_crs(epsg=WGS84_EPSG)
    logger.info("Starting OSM queries...")
    features_list = parallel_osm_queries(buffers_wgs)
    points_gdf = gpd.GeoDataFrame(
        {'distance_m': distances, 'features_nearby': features_list},
        geometry=points,
        crs=route_gdf.crs
    )
    geojson_output = OUTPUT_DIR / 'train_route_100m_linearbuffer.geojson'
    csv_output = OUTPUT_DIR / 'train_route_100m_linearbuffer.csv'
    save_outputs(points_gdf, geojson_output, csv_output)
    logger.info("\nResults:")
    logger.info(points_gdf[['distance_m', 'features_nearby']].to_string())
    logger.info(f"\n✔️ Done! Total runtime: {datetime.now() - start_time}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.error(f"Script failed: {e}")
        raise

In [2]:
import geopandas as gpd
import requests
import time
from datetime import datetime
import pandas as pd

# --- Start Timer ---
start_time = datetime.now()

# --- Load Train Route Shapefile ---
route_fp = r'خط مترو.shp'
route_gdf = gpd.read_file(route_fp)

# --- Reproject to UTM for accurate distances ---
utm_epsg = 32643  # Adjust for your region
if route_gdf.crs.to_epsg() != utm_epsg:
    route_gdf = route_gdf.to_crs(epsg=utm_epsg)

# --- Ensure geometry is LineString ---
route_line = route_gdf.geometry.iloc[0]
if route_line.geom_type != 'LineString':
    raise ValueError("The shapefile must contain a LineString geometry.")

# --- Generate Points Every 100m ---
def generate_points_by_meter(line, interval_m=100):
    points = []
    distances = []
    for i in range(int(line.length // interval_m) + 1):
        dist = i * interval_m
        point = line.interpolate(dist)
        points.append(point)
        distances.append(dist)
    return points, distances

points, distances = generate_points_by_meter(route_line, interval_m=100)

# --- Create GeoDataFrame of Points ---
points_gdf = gpd.GeoDataFrame({'distance_m': distances}, geometry=points, crs=route_gdf.crs)

# --- Create Linear Buffers (1km wide: 500m each side) ---
buffer_width = 1000  # meters
buffers = points_gdf.buffer(buffer_width / 2)  # 500m each side

# --- Convert Buffers to Centroids for Overpass API ---
buffer_centroids = buffers.centroid
buffers_gdf = gpd.GeoDataFrame({'distance_m': distances}, geometry=buffer_centroids, crs=route_gdf.crs)

# --- Convert to WGS84 for OSM Queries ---
buffers_wgs = buffers_gdf.to_crs(epsg=4326)

# --- OSM Query Function ---
def query_osm_features(point, radius=1000):
    lat, lon = point.y, point.x
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      node["place"~"village|hamlet|town"](around:{radius},{lat},{lon});
      way["natural"="water"](around:{radius},{lat},{lon});
      way["waterway"="river"](around:{radius},{lat},{lon});
      relation["natural"="water"](around:{radius},{lat},{lon});
      way["highway"](around:{radius},{lat},{lon});
      way["leisure"~"park|garden"](around:{radius},{lat},{lon});
      way["landuse"="grass"](around:{radius},{lat},{lon});
    );
    out center;
    """
    try:
        response = requests.get(overpass_url, params={'data': query})
        time.sleep(1.2)
        if response.status_code != 200:
            return ["[Error] No response"]
        data = response.json()
        features = []
        for el in data['elements']:
            tags = el.get('tags', {})
            name = tags.get('name', 'Unnamed')
            if 'place' in tags:
                features.append(f"Place: {name}")
            elif tags.get('waterway') == 'river':
                features.append(f"River: {name}")
            elif tags.get('natural') == 'water':
                features.append(f"Water body: {name}")
            elif tags.get('highway'):
                features.append(f"Road: {name}")
            elif tags.get('leisure') in ['park', 'garden']:
                features.append(f"{tags['leisure'].capitalize()}: {name}")
            elif tags.get('landuse') == 'grass':
                features.append(f"Green area: {name}")
        return features if features else ["None"]
    except Exception as e:
        return [f"[Error] {e}"]

# --- Run Overpass Queries ---
features_list = []
for idx, row in buffers_wgs.iterrows():
    point = row.geometry
    features = query_osm_features(point, radius=1000)
    features_list.append("; ".join(features))

# --- Merge with original points_gdf ---
points_gdf['features_nearby'] = features_list

# --- Save Outputs ---
geojson_output = 'train_route_100m_linearbuffer.geojson'
csv_output = 'train_route_100m_linearbuffer.csv'

points_gdf.to_file(geojson_output, driver='GeoJSON')
points_gdf[['distance_m', 'features_nearby']].to_csv(csv_output, index=False, encoding='utf-8-sig')

# --- Print Results and Runtime ---
print(points_gdf[['distance_m', 'features_nearby']])
print(f"\n✔️ Done! Total runtime: {datetime.now() - start_time}")


     distance_m                                    features_nearby
0             0  Road: بزرگراه وکیل آباد; Road: بزرگراه وکیل آب...
1           100  Road: بزرگراه وکیل آباد; Road: بزرگراه وکیل آب...
2           200  Road: بزرگراه وکیل آباد; Road: بزرگراه وکیل آب...
3           300  Road: بزرگراه وکیل آباد; Road: بزرگراه وکیل آب...
4           400  Road: بزرگراه وکیل آباد; Road: بولوار امام رضا...
..          ...                                                ...
451       45100  Road: بلوار امام رضا; Road: بلوار امام رضا; Ro...
452       45200  Road: بلوار امام رضا; Road: بلوار امام رضا; Ro...
453       45300  Road: بلوار امام رضا; Road: بلوار امام رضا; Ro...
454       45400  Road: بلوار امام رضا; Road: بلوار امام رضا; Ro...
455       45500  Road: بلوار امام رضا; Road: بلوار امام رضا; Ro...

[456 rows x 2 columns]

✔️ Done! Total runtime: 0:30:29.240885
